In [14]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)



features = ['ADAS_Q1', 'ADAS_Q4', 'LDELTOTAL', 'AVDEL30MIN','tmab_time', 
           'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
           'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg', 'faq1','faq2','faq3','faq4','faq5','faq6','faq7','faq8','faq9','faq10']

categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({1:0, 2:1},inplace=True)

In [ ]:
# Divide data in half for tuning, to reduce leakage when using split/repeat cross-validation
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)

from imblearn.over_sampling import SMOTENC
sm = SMOTENC(categorical_features=categorical_features, sampling_strategy=.5, random_state=33433)
X, y = sm.fit_resample(X,y)
y = pd.Series(y)   

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 4.43 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.976902,0.969565,0.974457,0.973234,0.969565,0.976902,0.974457,0.973234,0.969565,0.976902,0.974457,0.973234
std,0.007455,0.012011,0.006902,0.007598,0.012011,0.007455,0.006902,0.007598,0.012011,0.007455,0.006902,0.007598
min,0.967391,0.945652,0.960145,0.956522,0.945652,0.967391,0.960145,0.956522,0.945652,0.967391,0.960145,0.956522
25%,0.971467,0.964674,0.970109,0.970788,0.964674,0.971467,0.970109,0.970788,0.964674,0.971467,0.970109,0.970788
50%,0.975543,0.967391,0.978261,0.975543,0.967391,0.975543,0.978261,0.975543,0.967391,0.975543,0.978261,0.975543
75%,0.983696,0.978261,0.978261,0.978940,0.978261,0.983696,0.978261,0.978940,0.978261,0.983696,0.978261,0.978940
max,0.989130,0.989130,0.981884,0.980978,0.989130,0.989130,0.981884,0.980978,0.989130,0.989130,0.981884,0.980978


## Random Search

In [6]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  6.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=0.5,
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                  

In [8]:
rf_search_rand.best_params_

{'n_estimators': 400,
 'min_samples_split': 10,
 'min_samples_leaf': 2,
 'max_features': 0.3,
 'max_depth': 60,
 'bootstrap': False}

In [9]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.978261,0.950483,0.969002,0.964372,0.950483,0.978261,0.969002,0.964372,0.950483,0.978261,0.969002,0.964372
std,0.004707,0.029323,0.007271,0.012705,0.029323,0.004707,0.007271,0.012705,0.029323,0.004707,0.007271,0.012705
min,0.972826,0.923913,0.960145,0.951087,0.923913,0.972826,0.960145,0.951087,0.923913,0.972826,0.960145,0.951087
25%,0.972826,0.934783,0.963768,0.956522,0.934783,0.972826,0.963768,0.956522,0.934783,0.972826,0.963768,0.956522
50%,0.978261,0.934783,0.967391,0.959239,0.934783,0.978261,0.967391,0.959239,0.934783,0.978261,0.967391,0.959239
75%,0.983696,0.989130,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978
max,0.983696,0.989130,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978


## Grid Search

In [10]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [300, 400, 500],
               'max_features': [.2, .3, .4],
               'max_depth': [50, 60, 70],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [1, 2, 3],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:   37.0s
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  2.9min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=0.5,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=555,
                                    

In [11]:
rf_search_grid.best_params_

{'bootstrap': False,
 'max_depth': 50,
 'max_features': 0.3,
 'min_samples_leaf': 2,
 'min_samples_split': 9,
 'n_estimators': 300}

In [12]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.978261,0.951691,0.969404,0.964976,0.951691,0.978261,0.969404,0.964976,0.951691,0.978261,0.969404,0.964976
std,0.004707,0.028298,0.007042,0.012262,0.028298,0.004707,0.007042,0.012262,0.028298,0.004707,0.007042,0.012262
min,0.972826,0.923913,0.960145,0.951087,0.923913,0.972826,0.960145,0.951087,0.923913,0.972826,0.960145,0.951087
25%,0.972826,0.934783,0.963768,0.956522,0.934783,0.972826,0.963768,0.956522,0.934783,0.972826,0.963768,0.956522
50%,0.978261,0.934783,0.967391,0.959239,0.934783,0.978261,0.967391,0.959239,0.934783,0.978261,0.967391,0.959239
75%,0.983696,0.989130,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978
max,0.983696,0.989130,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978,0.989130,0.983696,0.978261,0.980978


In [13]:
rf = RandomForestClassifier(n_estimators=300, max_features=.3, max_depth = 50, min_samples_split = 9, min_samples_leaf = 2, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 10.9 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.976902,0.972826,0.975543,0.974864,0.972826,0.976902,0.975543,0.974864,0.972826,0.976902,0.975543,0.974864
std,0.010263,0.012468,0.007697,0.007730,0.012468,0.010263,0.007697,0.007730,0.012468,0.010263,0.007697,0.007730
min,0.967391,0.956522,0.963768,0.961957,0.956522,0.967391,0.963768,0.961957,0.956522,0.967391,0.963768,0.961957
25%,0.967391,0.964674,0.971920,0.974185,0.964674,0.967391,0.971920,0.974185,0.964674,0.967391,0.971920,0.974185
50%,0.972826,0.972826,0.976449,0.978261,0.972826,0.972826,0.976449,0.978261,0.972826,0.972826,0.976449,0.978261
75%,0.985054,0.980978,0.981884,0.978940,0.980978,0.985054,0.981884,0.978940,0.980978,0.985054,0.981884,0.978940
max,0.994565,0.989130,0.985507,0.980978,0.989130,0.994565,0.985507,0.980978,0.989130,0.994565,0.985507,0.980978


In [15]:
# All samples
rf = RandomForestClassifier(n_estimators=300, max_features=.3, max_depth = 50, min_samples_split = 9, min_samples_leaf = 2, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(10)),repeats=list(int(x)*42+42 for x in range(10)), imbalanced='over', categorical_features=categorical_features, initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 58.9 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.989322,0.616957,0.967474,0.803140,0.616957,0.989322,0.967474,0.803140,0.616957,0.989322,0.967474,0.803140
std,0.007546,0.082145,0.008239,0.040926,0.082145,0.007546,0.008239,0.040926,0.082145,0.007546,0.008239,0.040926
min,0.970190,0.478261,0.946429,0.733710,0.478261,0.970190,0.946429,0.733710,0.478261,0.970190,0.946429,0.733710
25%,0.985772,0.565217,0.961735,0.769059,0.565217,0.985772,0.961735,0.769059,0.565217,0.985772,0.961735,0.769059
50%,0.991870,0.608696,0.969388,0.801638,0.608696,0.991870,0.969388,0.801638,0.608696,0.991870,0.969388,0.801638
75%,0.994580,0.695652,0.974490,0.841051,0.695652,0.994580,0.974490,0.841051,0.695652,0.994580,0.974490,0.841051
max,1.000000,0.782609,0.979592,0.883174,0.782609,1.000000,0.979592,0.883174,0.782609,1.000000,0.979592,0.883174


# Outputs

In [16]:
### Save outputs ###
q = 'CDR1_BorutaECOGFAQ_2'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)